# Healthcare Analytics: Delta Liquid Clustering Demo

## Overview

This notebook demonstrates the power of **Delta Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a healthcare analytics use case. Liquid clustering is a revolutionary feature that automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Patient Diagnosis Analytics

We'll analyze patient diagnosis records from a healthcare system. Our clustering strategy will optimize for:
- **Patient-specific queries**: Fast lookups by patient ID
- **Time-based analysis**: Efficient filtering by diagnosis date
- **Diagnosis patterns**: Quick aggregation by diagnosis type

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [ ]:
# Create healthcare catalog and gold schema
# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS healthcare")
spark.sql("CREATE SCHEMA IF NOT EXISTS healthcare.gold")

print("Healthcare catalog and gold schema created successfully!")

## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `patient_diagnoses` table will store:
- **patient_id**: Unique patient identifier
- **diagnosis_date**: When the diagnosis was made
- **diagnosis_code**: ICD-10 diagnosis code
- **diagnosis_description**: Human-readable diagnosis
- **severity_level**: Critical, High, Medium, Low
- **treating_physician**: Physician ID
- **facility_id**: Healthcare facility

### Clustering Strategy

We'll cluster by `patient_id` and `diagnosis_date` because:
- **patient_id**: Patients often have multiple visits, grouping their records together
- **diagnosis_date**: Time-based queries are common in healthcare analytics
- This combination optimizes for both patient history lookups and temporal analysis

In [ ]:
# Create Delta table with liquid clustering
# CLUSTER BY defines the columns for automatic optimization

spark.sql("""
CREATE TABLE IF NOT EXISTS healthcare.gold.patient_diagnoses (
    patient_id STRING,
    diagnosis_date DATE,
    diagnosis_code STRING,
    diagnosis_description STRING,
    severity_level STRING,
    treating_physician STRING,
    facility_id STRING
)
USING DELTA
CLUSTER BY (patient_id, diagnosis_date)
""")

print("Delta table with liquid clustering created successfully!")
print("Clustering will automatically optimize data layout for queries on patient_id and diagnosis_date.")

Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on patient_id and diagnosis_date.


## Step 3: Generate Healthcare Sample Data

### Data Generation Strategy

We'll create realistic healthcare data including:
- **100 patients** with multiple diagnoses over time
- **Common diagnoses**: Diabetes, Hypertension, Asthma, etc.
- **Realistic temporal patterns**: Follow-up visits, chronic condition management
- **Multiple facilities**: Different hospitals/clinics

### Why This Data Pattern?

This data simulates real healthcare scenarios where:
- Patients have multiple encounters
- Chronic conditions require ongoing monitoring
- Time-based analysis reveals treatment effectiveness
- Facility-level reporting is needed

In [ ]:
# Generate sample healthcare diagnosis data
# Using fully qualified pyspark.sql.functions to avoid conflicts

import random
from datetime import datetime, timedelta

# Define healthcare data constants
DIAGNOSES = [
    ("E11.9", "Type 2 diabetes mellitus without complications", "Medium"),
    ("I10", "Essential hypertension", "High"),
    ("J45.909", "Unspecified asthma, uncomplicated", "Medium"),
    ("M54.5", "Low back pain", "Low"),
    ("N39.0", "Urinary tract infection, site not specified", "Medium"),
    ("Z51.11", "Encounter for antineoplastic chemotherapy", "Critical"),
    ("I25.10", "Atherosclerotic heart disease of native coronary artery without angina pectoris", "High"),
    ("F41.9", "Anxiety disorder, unspecified", "Medium"),
    ("M79.3", "Panniculitis, unspecified", "Low"),
    ("Z00.00", "Encounter for general adult medical examination without abnormal findings", "Low")
]

FACILITIES = ["HOSP001", "HOSP002", "CLINIC001", "CLINIC002", "URGENT001"]
PHYSICIANS = ["DR_SMITH", "DR_JOHNSON", "DR_WILLIAMS", "DR_BROWN", "DR_JONES", "DR_GARCIA", "DR_MILLER", "DR_DAVIS"]

# Generate patient diagnosis records
patient_data = []
base_date = datetime(2024, 1, 1)

# Create 100 patients with 2-5 diagnoses each
for patient_num in range(1, 101):
    patient_id = f"PAT{patient_num:04d}"
    
    # Each patient gets 2-5 diagnoses over several months
    num_diagnoses = random.randint(2, 5)
    
    for i in range(num_diagnoses):
        # Spread diagnoses over 6 months
        days_offset = random.randint(0, 180)
        diagnosis_date = base_date + timedelta(days=days_offset)
        
        # Select random diagnosis
        diagnosis_code, description, severity = random.choice(DIAGNOSES)
        
        # Select random facility and physician
        facility = random.choice(FACILITIES)
        physician = random.choice(PHYSICIANS)
        
        patient_data.append({
            "patient_id": patient_id,
            "diagnosis_date": diagnosis_date.date(),
            "diagnosis_code": diagnosis_code,
            "diagnosis_description": description,
            "severity_level": severity,
            "treating_physician": physician,
            "facility_id": facility
        })

print(f"Generated {len(patient_data)} patient diagnosis records")
print("Sample record:", patient_data[0])

Generated 350 patient diagnosis records
Sample record: {'patient_id': 'PAT0001', 'diagnosis_date': datetime.date(2024, 2, 17), 'diagnosis_code': 'F41.9', 'diagnosis_description': 'Anxiety disorder, unspecified', 'severity_level': 'Medium', 'treating_physician': 'DR_SMITH', 'facility_id': 'CLINIC002'}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:
1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [ ]:
# Insert data using PySpark DataFrame operations
# Using fully qualified function references to avoid conflicts

# Create DataFrame from generated data
df_diagnoses = spark.createDataFrame(patient_data)

# Display schema and sample data
print("DataFrame Schema:")
df_diagnoses.printSchema()

print("\nSample Data:")
df_diagnoses.show(5)

# Insert data into Delta table with liquid clustering
# The CLUSTER BY (patient_id, diagnosis_date) will automatically optimize the data layout
df_diagnoses.write.mode("overwrite").saveAsTable("healthcare.gold.patient_diagnoses")

print(f"\nSuccessfully inserted {df_diagnoses.count()} records into healthcare.gold.patient_diagnoses")
print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- diagnosis_code: string (nullable = true)
 |-- diagnosis_date: date (nullable = true)
 |-- diagnosis_description: string (nullable = true)
 |-- facility_id: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- severity_level: string (nullable = true)
 |-- treating_physician: string (nullable = true)


Sample Data:
+--------------+--------------+---------------------+-----------+----------+--------------+------------------+
|diagnosis_code|diagnosis_date|diagnosis_description|facility_id|patient_id|severity_level|treating_physician|
+--------------+--------------+---------------------+-----------+----------+--------------+------------------+
|         F41.9|    2024-02-17| Anxiety disorder,...|  CLINIC002|   PAT0001|        Medium|          DR_SMITH|
|           I10|    2024-01-15| Essential hyperte...|    HOSP002|   PAT0001|          High|        DR_JOHNSON|
|       J45.909|    2024-02-13| Unspecified asthm...|    HOSP002|   PAT0001|       


Successfully inserted 350 records into healthcare.gold.patient_diagnoses
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Patient history lookup** (clustered by patient_id)
2. **Time-based analysis** (clustered by diagnosis_date)
3. **Combined patient + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:
- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [ ]:
# Demonstrate liquid clustering benefits with optimized queries

# Query 1: Patient history - benefits from patient_id clustering
print("=== Query 1: Patient Diagnosis History ===")
patient_history = spark.sql("""
SELECT patient_id, diagnosis_date, diagnosis_code, diagnosis_description, severity_level
FROM healthcare.gold.patient_diagnoses
WHERE patient_id = 'PAT0001'
ORDER BY diagnosis_date
""")

patient_history.show()
print(f"Records found: {patient_history.count()}")

# Query 2: Time-based analysis - benefits from diagnosis_date clustering
print("\n=== Query 2: Recent Critical Diagnoses ===")
recent_critical = spark.sql("""
SELECT diagnosis_date, patient_id, diagnosis_code, diagnosis_description, treating_physician
FROM healthcare.gold.patient_diagnoses
WHERE diagnosis_date >= '2024-04-01' AND severity_level = 'Critical'
ORDER BY diagnosis_date DESC
""")

recent_critical.show()
print(f"Critical diagnoses found: {recent_critical.count()}")

# Query 3: Combined patient + time query - optimal for our clustering strategy
print("\n=== Query 3: Patient Timeline Analysis ===")
patient_timeline = spark.sql("""
SELECT patient_id, diagnosis_date, diagnosis_code, severity_level, facility_id
FROM healthcare.gold.patient_diagnoses
WHERE patient_id LIKE 'PAT001%' AND diagnosis_date >= '2024-03-01'
ORDER BY patient_id, diagnosis_date
""")

patient_timeline.show()
print(f"Timeline records found: {patient_timeline.count()}")

=== Query 1: Patient Diagnosis History ===


+----------+--------------+--------------+---------------------+--------------+
|patient_id|diagnosis_date|diagnosis_code|diagnosis_description|severity_level|
+----------+--------------+--------------+---------------------+--------------+
|   PAT0001|    2024-01-15|           I10| Essential hyperte...|          High|
|   PAT0001|    2024-02-13|       J45.909| Unspecified asthm...|        Medium|
|   PAT0001|    2024-02-17|         F41.9| Anxiety disorder,...|        Medium|
+----------+--------------+--------------+---------------------+--------------+



Records found: 3

=== Query 2: Recent Critical Diagnoses ===


+--------------+----------+--------------+---------------------+------------------+
|diagnosis_date|patient_id|diagnosis_code|diagnosis_description|treating_physician|
+--------------+----------+--------------+---------------------+------------------+
|    2024-06-25|   PAT0061|        Z51.11| Encounter for ant...|       DR_WILLIAMS|
|    2024-06-24|   PAT0099|        Z51.11| Encounter for ant...|         DR_GARCIA|
|    2024-06-19|   PAT0082|        Z51.11| Encounter for ant...|          DR_BROWN|
|    2024-06-18|   PAT0018|        Z51.11| Encounter for ant...|          DR_DAVIS|
|    2024-06-16|   PAT0091|        Z51.11| Encounter for ant...|       DR_WILLIAMS|
|    2024-06-05|   PAT0056|        Z51.11| Encounter for ant...|        DR_JOHNSON|
|    2024-06-03|   PAT0042|        Z51.11| Encounter for ant...|          DR_JONES|
|    2024-05-31|   PAT0062|        Z51.11| Encounter for ant...|          DR_SMITH|
|    2024-05-24|   PAT0023|        Z51.11| Encounter for ant...|          DR

Critical diagnoses found: 21

=== Query 3: Patient Timeline Analysis ===


+----------+--------------+--------------+--------------+-----------+
|patient_id|diagnosis_date|diagnosis_code|severity_level|facility_id|
+----------+--------------+--------------+--------------+-----------+
|   PAT0010|    2024-05-05|         E11.9|        Medium|    HOSP002|
|   PAT0010|    2024-05-21|         M79.3|           Low|  URGENT001|
|   PAT0010|    2024-06-28|         M54.5|           Low|    HOSP002|
|   PAT0011|    2024-03-09|         F41.9|        Medium|    HOSP002|
|   PAT0011|    2024-03-29|         N39.0|        Medium|  CLINIC001|
|   PAT0012|    2024-04-14|         M54.5|           Low|  URGENT001|
|   PAT0012|    2024-04-17|         M79.3|           Low|  CLINIC002|
|   PAT0012|    2024-06-03|           I10|          High|    HOSP002|
|   PAT0013|    2024-06-18|         E11.9|        Medium|  CLINIC001|
|   PAT0014|    2024-04-04|       J45.909|        Medium|    HOSP001|
|   PAT0014|    2024-05-13|         N39.0|        Medium|    HOSP002|
|   PAT0014|    2024

Timeline records found: 25


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the healthcare insights possible with this optimized structure.

### Key Analytics

- **Diagnosis frequency** by type
- **Severity distribution** across facilities
- **Physician workload** analysis
- **Temporal patterns** in diagnoses

In [ ]:
# Analyze clustering effectiveness and healthcare insights

# Diagnosis frequency analysis
print("=== Diagnosis Frequency Analysis ===")
diagnosis_freq = spark.sql("""
SELECT diagnosis_code, diagnosis_description, COUNT(*) as frequency,
       ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
FROM healthcare.gold.patient_diagnoses
GROUP BY diagnosis_code, diagnosis_description
ORDER BY frequency DESC
""")

diagnosis_freq.show(truncate=False)

# Severity distribution by facility
print("\n=== Severity Distribution by Facility ===")
severity_by_facility = spark.sql("""
SELECT facility_id, severity_level, COUNT(*) as count
FROM healthcare.gold.patient_diagnoses
GROUP BY facility_id, severity_level
ORDER BY facility_id, severity_level
""")

severity_by_facility.show()

# Physician workload analysis
print("\n=== Physician Workload Analysis ===")
physician_workload = spark.sql("""
SELECT treating_physician, COUNT(*) as total_diagnoses,
       COUNT(DISTINCT patient_id) as unique_patients,
       ROUND(AVG(CASE WHEN severity_level = 'Critical' THEN 1 ELSE 0 END), 3) as critical_case_ratio
FROM healthcare.gold.patient_diagnoses
GROUP BY treating_physician
ORDER BY total_diagnoses DESC
""")

physician_workload.show()

=== Diagnosis Frequency Analysis ===


+--------------+-------------------------------------------------------------------------------+---------+----------+
|diagnosis_code|diagnosis_description                                                          |frequency|percentage|
+--------------+-------------------------------------------------------------------------------+---------+----------+
|Z00.00        |Encounter for general adult medical examination without abnormal findings      |43       |12.29     |
|N39.0         |Urinary tract infection, site not specified                                    |40       |11.43     |
|M54.5         |Low back pain                                                                  |40       |11.43     |
|Z51.11        |Encounter for antineoplastic chemotherapy                                      |38       |10.86     |
|J45.909       |Unspecified asthma, uncomplicated                                              |37       |10.57     |
|F41.9         |Anxiety disorder, unspecified           

+-----------+--------------+-----+
|facility_id|severity_level|count|
+-----------+--------------+-----+
|  CLINIC001|      Critical|    8|
|  CLINIC001|          High|    8|
|  CLINIC001|           Low|   20|
|  CLINIC001|        Medium|   25|
|  CLINIC002|      Critical|    8|
|  CLINIC002|          High|    9|
|  CLINIC002|           Low|   24|
|  CLINIC002|        Medium|   23|
|    HOSP001|      Critical|    6|
|    HOSP001|          High|    8|
|    HOSP001|           Low|   18|
|    HOSP001|        Medium|   30|
|    HOSP002|      Critical|   10|
|    HOSP002|          High|   14|
|    HOSP002|           Low|   27|
|    HOSP002|        Medium|   33|
|  URGENT001|      Critical|    6|
|  URGENT001|          High|   14|
|  URGENT001|           Low|   24|
|  URGENT001|        Medium|   35|
+-----------+--------------+-----+


=== Physician Workload Analysis ===


+------------------+---------------+---------------+-------------------+
|treating_physician|total_diagnoses|unique_patients|critical_case_ratio|
+------------------+---------------+---------------+-------------------+
|          DR_BROWN|             57|             45|              0.123|
|          DR_DAVIS|             56|             42|              0.089|
|          DR_SMITH|             47|             38|               0.17|
|         DR_GARCIA|             45|             38|              0.133|
|       DR_WILLIAMS|             40|             30|              0.075|
|         DR_MILLER|             38|             33|              0.079|
|        DR_JOHNSON|             37|             35|              0.108|
|          DR_JONES|             30|             27|              0.067|
+------------------+---------------+---------------+-------------------+



## Key Takeaways: Delta Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `CLUSTER BY (patient_id, diagnosis_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (patient_id, diagnosis_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Healthcare analytics where patient history lookups and temporal analysis are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for healthcare data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles healthcare-scale data volumes effortlessly

### Best Practices for Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger healthcare datasets
- Integrate with real healthcare systems

This notebook demonstrates how Oracle AI Data Platform makes advanced analytics accessible while maintaining enterprise-grade performance and governance.